In [1]:
import pandas as pd
from transformers import pipeline
from datasets import load_dataset
from datasets import Dataset
from tqdm.auto import tqdm
import torch

c:\Users\phku0\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# pipe = pipeline(
#   "text-classification",
#   model="kk08/CryptoBERT",
#   truncation=True,
#   max_length=512
# )

# pipe("Bitcoin is a scam. Million of people have lost their money. The SEC decided to ban it.")

In [3]:
# pipe = pipeline(
#   "text-classification",
#   model="ProsusAI/finbert",
#   truncation=True,
#   max_length=512
# )

# pipe("Bitcoin is a scam. Million of people have lost their money. The SEC decided to ban it.")

In [4]:
df = pd.read_parquet("../data/4c.cryptonews_ner_spacy.parquet")
print(df.shape)
df.head()

(159486, 11)


,title,text,source_name,date,tanalysis_absa,economy_absa,regulation_absa,technology_absa,adoption_absa,cybersecurity_absa,NER
12,VanEck makes another Bitcoin ETF run,Investment management firm VanEck has filed wi...,Modern Consensus,"Fri, 01 Jan 2021 11:07:58 -0500",0.0,0.0,0.3,0.0,0.7,0.0,"[[VanEck, ORG], [VanEck, ORG], [the U.S. Secur..."
16,Markets Report: Bitcoin starts 2021 with a ban...,Bitcoin begins a new year on a high in almost ...,Modern Consensus,"Fri, 01 Jan 2021 09:51:10 -0500",0.9,0.0,0.0,0.0,0.5,0.0,"[[2021, DATE], [500,000, MONEY], [a new year, ..."
87,Bitcoin Rally Extends Into New Year,Jan.03 -- Bitcoin is continuing its rally into...,Bloomberg Markets and Finance,"Sun, 03 Jan 2021 18:46:20 -0500",0.8,0.0,0.0,0.0,0.0,0.0,"[[New Year, EVENT], [Jan.03 --, DATE], [the ne..."
127,"BitPay CCO Says Bitcoin Will Hit $45,000",Jan.04 -- Bitpay Chief Commercial Officer Sonn...,Bloomberg Technology,"Mon, 04 Jan 2021 20:29:35 -0500",0.8,0.0,0.0,0.0,0.0,0.0,"[[BitPay CCO, ORG], [45,000, MONEY], [Jan.04, ..."
166,"Markets Report: Bitcoin Dives to $27,700 as a ...",Bitcoin is seeing a wild start to the week wit...,Modern Consensus,"Mon, 04 Jan 2021 09:01:49 -0500",-0.8,0.0,0.0,0.0,0.0,0.0,"[[27,700, MONEY], [a Crazy Weekend, DATE], [th..."


In [5]:
dataset = Dataset.from_pandas(df)

dataset = dataset.map(lambda example: {
  'news_text_title': example['title'] + ' ' + example['text']
})

Map: 100%|██████████| 159486/159486 [00:15<00:00, 10101.22 examples/s]


In [6]:
pipe = pipeline(
  "text-classification",
  model="kk08/CryptoBERT",
  truncation=True,
  max_length=512
)

def classify_sentiment(batch):
  results = pipe(batch['news_text_title'])
  batch['sentiment_CryptoBERT'] = [result['label'] for result in results]
  batch['score_CryptoBERT'] = [result['score'] for result in results]
  return batch

dataset = dataset.map(classify_sentiment, batched=True, batch_size=5)

Device set to use cuda:0
Map: 100%|██████████| 159486/159486 [24:43<00:00, 107.49 examples/s]


In [7]:
pipe = pipeline(
  "text-classification",
  model="ProsusAI/finbert",
  truncation=True,
  max_length=512
)

def classify_sentiment(batch):
  results = pipe(batch['news_text_title'])
  batch['sentiment_finbert'] = [result['label'] for result in results]
  batch['score_finbert'] = [result['score'] for result in results]
  return batch

dataset = dataset.map(classify_sentiment, batched=True, batch_size=5)

Device set to use cuda:0
Map: 100%|██████████| 159486/159486 [23:45<00:00, 111.89 examples/s]


In [8]:
df = dataset.to_pandas()
df.set_index('__index_level_0__', inplace=True)

print(df.shape)
df.head()

(159486, 16)


,title,text,source_name,date,tanalysis_absa,economy_absa,regulation_absa,technology_absa,adoption_absa,cybersecurity_absa,NER,news_text_title,sentiment_CryptoBERT,score_CryptoBERT,sentiment_finbert,score_finbert
__index_level_0__,,,,,,,,,,,,,,,,
12,VanEck makes another Bitcoin ETF run,Investment management firm VanEck has filed wi...,Modern Consensus,"Fri, 01 Jan 2021 11:07:58 -0500",0.0,0.0,0.3,0.0,0.7,0.0,"[[VanEck, ORG], [VanEck, ORG], [the U.S. Secur...",VanEck makes another Bitcoin ETF run Investmen...,LABEL_1,0.953463,neutral,0.704972
16,Markets Report: Bitcoin starts 2021 with a ban...,Bitcoin begins a new year on a high in almost ...,Modern Consensus,"Fri, 01 Jan 2021 09:51:10 -0500",0.9,0.0,0.0,0.0,0.5,0.0,"[[2021, DATE], [500,000, MONEY], [a new year, ...",Markets Report: Bitcoin starts 2021 with a ban...,LABEL_1,0.966029,positive,0.927262
87,Bitcoin Rally Extends Into New Year,Jan.03 -- Bitcoin is continuing its rally into...,Bloomberg Markets and Finance,"Sun, 03 Jan 2021 18:46:20 -0500",0.8,0.0,0.0,0.0,0.0,0.0,"[[New Year, EVENT], [Jan.03 --, DATE], [the ne...",Bitcoin Rally Extends Into New Year Jan.03 -- ...,LABEL_1,0.957252,positive,0.817353
127,"BitPay CCO Says Bitcoin Will Hit $45,000",Jan.04 -- Bitpay Chief Commercial Officer Sonn...,Bloomberg Technology,"Mon, 04 Jan 2021 20:29:35 -0500",0.8,0.0,0.0,0.0,0.0,0.0,"[[BitPay CCO, ORG], [45,000, MONEY], [Jan.04, ...","BitPay CCO Says Bitcoin Will Hit $45,000 Jan.0...",LABEL_1,0.914190,neutral,0.362224
166,"Markets Report: Bitcoin Dives to $27,700 as a ...",Bitcoin is seeing a wild start to the week wit...,Modern Consensus,"Mon, 04 Jan 2021 09:01:49 -0500",-0.8,0.0,0.0,0.0,0.0,0.0,"[[27,700, MONEY], [a Crazy Weekend, DATE], [th...","Markets Report: Bitcoin Dives to $27,700 as a ...",LABEL_0,0.917384,negative,0.932793


In [10]:
# rename index column
df.index.name = 'index'

In [11]:
df.to_parquet("../data/5.cryptonews_sentiment.parquet")